In [74]:
import pandas as pd
import numpy as np

import requests
import json
from bs4 import BeautifulSoup

import dill
import re
import time

First get a list of all the wine URLs.

In [75]:
base_url = 'http://www.snooth.com/wines/#action=search&hide_state=1&country=US&color[0]={}&entity=wine&store_front=0&search_page={}'

In [76]:
url_list = list()

for color_no,pg_range in zip([0, 1, 2], [range(1,1001), range(1,1001), range(1,468)]):
    
    print color_no
    
    if color_no==0:
        color='red'
    elif color_no==1:
        color='white'
    else:
        color='rose'
    
    for pg in pg_range:
        url = base_url.format(color_no, pg)
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'lxml')

        the_list = [x.find('a', href=True).get('href') 
                         for x in soup.find_all(attrs={'class':'wine-name'}) 
                         if x.find('a', href=True) is not None]
        the_list = [(color,x) for x in the_list]
        url_list.extend(the_list)

0
1
2


In [110]:
len(url_list)

49340

In [111]:
with open('../pkl/02_snooth_dot_com_url_list.pkl','w') as fh:
    dill.dump(url_list, fh)

Then scrape the URLs.

In [112]:
proxy1 = {'https': 'socks5://127.0.0.1:8081'}
proxy2 = {'https': 'socks5://127.0.0.1:8082'}
proxy3 = {'https': 'socks5://127.0.0.1:8083'}
proxy4 = {'https': 'socks5://127.0.0.1:8084'}
proxy_list = [proxy1, proxy2, proxy3, proxy4]

In [ ]:
for url in enumerate(url_list):
    if (url[0] % 100) == 0:
        print url[0]
        
        if url[0] != 0:
            wine_df = pd.concat(wine_df_list, axis=0).reset_index(drop=True)
            wine_df.to_pickle('../pkl/02_snooth_dot_com_data_{}.pkl'.format(str(url[0])))
        
        wine_df_list = list()
        
    full_url = url[1][1]
    color = url[1][0]
    
    proxy = np.random.choice(proxy_list)
    html = requests.get(full_url, proxies=proxy)
    soup = BeautifulSoup(html.text, 'lxml')
    #time.sleep(1.5)
    
    wine_block1 = soup.find(attrs={'class':'wpp2014-wine_block-info'})
    wine_block2 = soup.find(attrs={'class':'wpp2014-reg_rat-region_vintage'})
    
    # wine name and year
    try:
        wine = wine_block1.find(attrs={'id':'wine-name'}).text.strip()
    except:
        wine = ''
        year = ''

    try:
        year = re.search(r"""((?:20|19)[0-9]{2})""", wine).group(1)
        wine = wine.replace(year, '').strip()
    except:
        year = ''

    # review
    try:
        review = wine_block1.find(attrs={'class':'winemakers-notes'}).text.replace("Winemaker's Notes:", '').strip()
    except:
        review = review.strip()
        
    # prices
    try:
        price_list = wine_block1.find(attrs={'class':'wpp2014-wine_block-sample_prices'}).find_all(attrs={'itemprop':'price'})
    except:
        price_list = np.NaN
    else:
        price_list = np.array([float(x.text) for x in price_list]).mean()
        
    
        
    # region, winery, varietal
    region = ''
    winery = ''
    varietal = ''

    try:
        data_list = [re.split(r"""\s+""", x.text.replace(u'\xbb', '').strip()) 
                     for x in wine_block2.find_all('div')]
    except:
        pass

    try:
        for l in data_list:
            if 'region' in l[0].lower():
                region = ' '.join(l[1:])
            if 'winery' in l[0].lower():
                winery = ' '.join(l[1:])
            if 'varietal' in l[0].lower():
                varietal = ' '.join(l[1:])
    except:
        pass
    
    
    df = pd.DataFrame({'wine':wine, 'year':year, 
                   'review':review,
                   'region':region, 'winery':winery, 'varietal':varietal,
                       'price':price_list,
                   'color':color, 'url':full_url}, index=pd.Index([0]))
    
    wine_df_list.append(df)
    
wine_df = pd.concat(wine_df_list, axis=0).reset_index(drop=True)
wine_df.to_pickle('../pkl/02_snooth_dot_com_data_{}.pkl'.format(str(url[0])))

0
100
200
300
400
